In [4]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

from scipy.io import mmread

sys.path.append(os.path.abspath("./src"))

import utility as ut
import hessenberg as hg
from variables import *
from qr import QR


pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 1000)
pd.set_option("display.precision", 12)
print("Setup compelete.")


Setup compelete.


# Hessenberg Transform

The Hessenberg transform is a similarity transformation used to reduce a complex matrix to hessenberg form. The following implementation performs the transform using *householder vectors*. The following cells check that for a given matrix $M \in \mathbb{C}^{n \times n}$,
$$M = UHU^{*}$$
where, H is the hessenberg form of $M$ and $U$ is a unitary matrix.

The unittests present in `/tests` check for the equivalence of the eigenvalues of $M$ and $H$. 

## Complex Matrices

In [5]:
a = -20
b = 50
n = 5
m = ut.complex_matrix(n, a, b)
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.conj().T - m, np.zeros((n, n)))}")

Original matrix:
                                   0                                 1                                 2                                 3                                 4
0  0.6696782689270-7.3815885064220j  12.506928117651+19.965456387162j  27.283831601984+25.294531237348j  10.970564438635+33.880362013856j  12.667215344132-18.801705706750j
1   47.752366110635+5.625320783373j -13.453914654475+48.048721433283j  -3.149339651258-18.487038707397j  27.158809235750+29.227676758519j  -7.216557618566+17.358487707208j
2 -9.3269502328750-3.2081775229380j  -2.632105720044-12.448959324186j  -3.859980375060+29.443471068737j   9.261707175312+35.906666021207j   6.372722904931+47.230686892053j
3   8.245367632522+41.374261009081j  18.101678361529-15.890059624227j   23.877388914371-0.134678604007j   26.166729057342-8.037097075136j   36.841495634809+1.876959460100j
4  32.204360096002+19.004606793642j -18.862080862263+22.487952688793j -5.7901998931870-3.4981606743450j  24.946119201799-1

## Real Matrices

### Random Matrices

In [6]:
a = -20
b = 50
n = 5
m = (b - a) * np.random.default_rng().random((n, n)) + a
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((n, n)))}")

Original matrix:
                  0                1                2                3                4
0  22.831855825716 -10.079418189274  34.529711575748  -8.471604503124  -3.278690548549
1 -18.557159243252  15.262143528205  30.329424604755  44.172035184412  22.435084714990
2  41.446789763873  -4.488100758293  31.906738755144  16.972696645508 -11.606781910368
3  -4.849153095586  44.760317268585  17.027664592377  21.310363990978   9.072739778005
4 -14.400921319147  18.091476245517   3.739266760629  40.386811811572  13.224686702591
Hessenberg transformed:
                  0                1                2                3                4
0  22.831855825716  35.636173520475  -4.434291098924   2.248202540874   9.042086690190
1  47.886379707541  27.728324413449   6.336783421836  19.141131551691 -14.949533232599
2   0.000000000000 -21.898055728191   1.471461532299  42.111973032001 -15.131931043302
3   0.000000000000   0.000000000000  41.829130088955  50.303342872025 -12.618043472148


### Matrix Market

In [7]:
files = ["west0381", "blckhole"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	h, u = hg.hessenberg_transform(m) 
	print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((m.shape[0], m.shape[0])))}")

Is the transformation similar: True
Is the transformation similar: True


# QR 

For a given matrix $M \in \mathbb{C}^{n\times n}$, in general, the $QR$ algorithm seeks to perform the following iteration:
* $Q_kR_k := M_k$
* $M_{k + 1} := R_kQ_k$

This algorithm can be made more stable and efficient in two ways. The first is to use $M$ is hessenberg form and the second is use to use shifts. When $H$ (hessenberg form of $M$), is used, the $QR$ decompisition of $H$ can be procedurally generated using Givens rotation matrices, $G$ (see the documentation for explanation and generation). The generation of the QR decomposition and then the subsequent formation of $RQ$ takes place as follows
* $R := G_1 G_2 \dots G_k H.$
* $Q := G_1 G_2 \dots G_k.$
* $H_{\text{new}} := R G_{k}^{*} G_{k - 1}^{*} \dots G_{1}^{*} = RQ.$

where $k \leq n - 2$.


## Wilkinson Shift

The Wilkinson shift employs stable shifts to accelerate convergence of the $QR$ hessenberg algorithm. In general the shifted algorithm looks as follows 
* $Q_kR_k := M_k - \sigma I$
* $M_{k + 1} := R_kQ_k + \sigma I$

The shift $\sigma$ is calculated as detailed in the documentation. The $QR$ decomposition and subsequent formation of $RQ$ is done using the hessenberg form of $M$ Givens matrices as shown above.

### Complex Matrices

In [8]:
a = -20
b = 50
n = 5
tol = 1e-8
m = ut.complex_matrix(n, a, b)
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")


                Eigenvalues (Numpy)
0  78.946486559668+52.821649792951j
1   11.155563036594-3.650871109134j
2  -9.585906349648+54.786619857393j
3 -21.339852275592-63.396944237597j
4 -51.508854830813+27.995642997399j
               Eigenvalues (Script)
0  78.946486559668+52.821649792951j
1   11.155563036572-3.650871109159j
2  -9.585906349626+54.786619857419j
3 -21.339852275592-63.396944237597j
4 -51.508854830812+27.995642997399j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


### Real Matrices

#### Random Matrices

In [14]:
a = -20
b = 50
n = 5
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 100)
eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0   83.869670863615+0.000000000000j
1  29.330356649093+16.228147596952j
2  29.330356649093-16.228147596952j
3  -21.698910577692+0.000000000000j
4  -38.477305285256+0.000000000000j
               Eigenvalues (Script)
0   83.869670863615+0.000000000000j
1  29.330356649093+16.228147596951j
2  29.330356649093-16.228147596951j
3  -21.698910577692+0.000000000000j
4  -38.477305285256+0.000000000000j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [13]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)
tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 100)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


#### Matrix Market

In [22]:
files = ["gre__115"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	tol = 1e-8
	qr_alg = QR(m)
	u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
	eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
	eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
	b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
	print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
	if not b:
		print(f"For matrix {file}")
		print(f"Number of mismatched eigenvalues: {mm.shape[0]}")
		print(f"Average absolute difference in mismatched values {mm['Difference'].mean()}")
		# with open("output_mm.txt", "w") as f:
		# 	f.write(f"{mm.to_string()}")
   


Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


## Double Shift (Inefficient)

#### Random Matrices

In [14]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)

tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 200)
eigs = np.sort(np.linalg.eig(m)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [4]:
a = -20
b = 50
n = 5
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 200)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0   51.419436746842-0.000000000000j
1  22.167285953793+32.133366474526j
2  22.167285953793-32.133366474526j
3  7.7293981794390+8.8070459431140j
4  7.7293981794390-8.8070459431140j
               Eigenvalues (Script)
0   51.419436746843+0.000000000000j
1  22.167285953793+32.133366474526j
2  22.167285953793-32.133366474526j
3  7.7293981794390+8.8070459431140j
4  7.7293981794390-8.8070459431140j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True
